## Hackathon 04: Performance Tuning
## Find Avg taxi-in (Flights) time per Description (Carrier)  per Airline Mfg (Planes)
### Take below query (run via RDD's) and Performance Tune it to see how fast you can get it to run

In [0]:
# First confirm have 3 files. If not notify instructor and he'll Email you the files
display(dbutils.fs.ls("dbfs:/FileStore/tables/header_flights.csv"))
display(dbutils.fs.ls("dbfs:/FileStore/tables/header_carriers.csv"))
display(dbutils.fs.ls("dbfs:/FileStore/tables/header_planes.csv"))

path,name,size
dbfs:/FileStore/tables/header_flights.csv,header_flights.csv,136035443


path,name,size
dbfs:/FileStore/tables/header_carriers.csv,header_carriers.csv,37794


path,name,size
dbfs:/FileStore/tables/header_planes.csv,header_planes.csv,432996


In [0]:
%scala

// First run as RDD to get benchmark of how much time to execute the query
val flight1 = sc.textFile("dbfs:/FileStore/tables/header_flights.csv").map(line => line.split(","))        
val carrier1 = sc.textFile("dbfs:/FileStore/tables/header_carriers.csv").map(line => line.split(","))        
val planes1 = sc.textFile("dbfs:/FileStore/tables/header_planes.csv").map(line => line.split(",")).filter(line => line.length == 9)               

val flight_header = flight1.first()
val carrier_header = carrier1.first()
val planes_header = planes1.first()

val flight2 = flight1.filter(x => x != flight_header) 
val carrier2 = carrier1.filter(x => x != carrier_header) 
val planes2 = planes1.filter(x => x != planes_header) 

val flightC = flight2.keyBy(line => line(5))
val carrierC = carrier2.keyBy(line => line(0))

flightC.first()
carrierC.first()

val join1 = flightC.join(carrierC)        
val join1P = join1.map{case (a,b) => (b._1(7), (a,b._1,b._2))} 

val planesP = planes2.keyBy(line => line(0))        
val finalRdd = join1P.join(planesP) 
finalRdd.first()
      
val neededData = finalRdd.map{case (a,b) => ((b._1._3(1), b._2(2)), b._1._2(10).toInt)}    
val avgTaxiInByAir = neededData.groupByKey().mapValues(list => list.sum.toFloat/list.size)   

avgTaxiInByAir.take(10)

flight1: org.apache.spark.rdd.RDD[Array[String]] = MapPartitionsRDD[12519] at map at command-4100678212116261:3
carrier1: org.apache.spark.rdd.RDD[Array[String]] = MapPartitionsRDD[12522] at map at command-4100678212116261:4
planes1: org.apache.spark.rdd.RDD[Array[String]] = MapPartitionsRDD[12526] at filter at command-4100678212116261:5
flight_header: Array[String] = Array(month, dayofmonth, dayofweek, deptime, arrtime, uniquecarrier, flightnum, tailnum, elapsedtime, airtime, arrdelay, depdelay, origin, dest, distance, taxiin, taxiout, cancelled, cancellationcode, diverted)
carrier_header: Array[String] = Array(Code, Description)
planes_header: Array[String] = Array(tailnum, type, manufacturer, issue_date, model, status, aircraft_type, engine_type, year)
flight2: org.apache.spark.rdd.RDD[Array[String]] = MapPartitionsRDD[12527] at filter at command-4100678212116261:11
carrier2: org.apache.spark.rdd.RDD[Array[String]] = MapPartitionsRDD[12528] at filter at command-4100678212116261:12
planes2: org.apache.spark.rdd.RDD[Array[String]] = MapPartitionsRDD[12529] at filter at command-4100678212116261:13
flightC: org.apache.spark.rdd.RDD[(String, Array[String])] = MapPartitionsRDD[12530] at keyBy at command-4100678212116261:15
carrierC: org.apache.spark.rdd.RDD[(String, Array[String])] = MapPartitionsRDD[12531] at keyBy at command-4100678212116261:16
join1: org.apache.spark.rdd.RDD[(String, (Array[String], Array[String]))] = MapPartitionsRDD[12534] at join at command-4100678212116261:21
join1P: org.apache.spark.rdd.RDD[(String, (String, Array[String], Array[String]))] = MapPartitionsRDD[12535] at map at command-4100678212116261:22
planesP: org.apache.spark.rdd.RDD[(String, Array[String])] = MapPartitionsRDD[12536] at keyBy at command-4100678212116261:24
finalRdd: org.apache.spark.rdd.RDD[(String, ((String, Array[String], Array[String]), Array[String]))] = MapPartitionsRDD[12539] at join at command-4100678212116261:25
neededData: org.apache.spark.rdd.RDD[((String, String), Int)] = MapPartitionsRDD[12540] at map at command-4100678212116261:28
avgTaxiInByAir: org.apache.spark.rdd.RDD[((String, String), Float)] = MapPartitionsRDD[12542] at mapValues at command-4100678212116261:29
res0: Array[((String, String), Float)] = Array(((Delta Air Lines Inc.,MCDONNELL DOUGLAS CORPORATION),8.265625), ((Alaska Airlines Inc.,MCDONNELL DOUGLAS),5.9397826), ((Southwest Airlines Co.,BOMBARDIER INC),5.275789), ((Mesa Airlines Inc.,BOEING),13.430187), ((Northwest Airlines Inc.,BOEING),7.36541), ((Southwest Airlines Co.,AIRBUS),7.5576377), ((American Eagle Airlines Inc.,PAIR MIKE E),-2.0257733), ((American Airlines Inc.,EMBRAER),10.972143), ((Frontier Airlines Inc.,MCDONNELL DOUGLAS AIRCRAFT CO),6.6797595), ((JetBlue Airways,BOEING),11.374466))

#### Re-write code in new Cell below.  When all code validates, do the following to get 'true' Timing for query
#### Terminate your Cluster.  Create new Cluster.  Attach Cluster to Notebook.  Run Notebook once.  Record timing for Cell

#  Deadline for email submission is Tuesday
## To save this Notebook as file, do the following:
  
#####1) In gray vertical bar on left, click on 'Workspace' icon
#####2) Find 'Hack-02a-Fly-Q' notebook, and click on 'down' arrow on right side of it
#####3) Select 'Export' > 'IPython Notebook'
#####4) Click 'Save'
#####5) Email file to following 2 email addresses: ottmk@ucmail.uc.edu;  XXX@mail.uc.edu;
#####   along with names of any team members so I can assign grades
#####6) You will be graded on 2 things.  I can run all the cells without Error and the Answer sets in Cells are correct.